# Тестирование Spell Checker

В этом ноутбуке будет произведено базовое тестировние написанной модели.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gc
import sys
import os
import json
import re
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer

from src.models.SpellChecker import *
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)

from IPython.display import display
from tqdm.notebook import tqdm

In [ ]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Инициализация

Начнем с того, что инициализируем все необходимые компоненты модели. Параллельно так же будет описана роль каждого компонента в системе.

In [ ]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

In [ ]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
handcode_table_path = os.path.join(DATA_PATH, 'processed', 'handcode_table', 
                                   'table.json')
with open(handcode_table_path, 'r') as inf:
    handcode_table = json.load(inf)
candidate_generator = CandidateGenerator(
    words=vocab.keys(), handcode_table=handcode_table, max_distance=1
)

In [ ]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_100.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_100.arpa.binary')
)
margin_border = np.log(2.5)
position_selector = KenlmMarginPositionSelector(
    model_left_right, model_right_left, margin_border=margin_border
)

In [ ]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
bert_scorer_correction = BertScorerCorrection(model, bert_tokenizer)
agg_subtoken_func = 'mean'
bert_scorer = BertScorer(
    bert_scorer_correction, agg_subtoken_func
)
candidate_scorer = CandidateScorer(bert_scorer)

In [ ]:
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    tokenizer,
    detokenizer,
    ignore_titles=True,
    max_it=max_it
)

## Тестирование

### Обучающая выборка

In [ ]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [ ]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) / batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [ ]:
!mkdir -p ../data/processed/results_base/

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'results_base', 'train.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [ ]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_base/diffs_train.txt ../data/external/spell_ru_eval/train_source.txt ../data/external/spell_ru_eval/train_corrected.txt ../data/processed/results_base/train.txt | tail -n 4

Как видим, имеем
* True Positive: $1092$
* Внесенных исправлений: $1911$ 
* Требуемых исправления: $1727$ 
* Precision: $57.14$, доверительный интервал: $(54.93, 59.36)$
* Recall: $63.23$, доверительный интервал: $(60.78, 65.68)$
* FMeasure: $60.03$, доверительный интервал: $(57.70, 62.36)$

### Тестовая выборка

In [ ]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'test_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'test_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [ ]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) / batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'results_base', 'test.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [ ]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_base/diffs_test.txt ../data/external/spell_ru_eval/test_source.txt ../data/external/spell_ru_eval/test_corrected.txt ../data/processed/results_base/test.txt | tail -n 4

Как видим, имеем
* True Positive: $1109$
* Внесенных исправлений: $2337$ 
* Требуемых исправления: $1976$ 
* Precision: $47.45$, доверительный интервал: $(45.42, 49.48)$
* Recall: $56.12$, доверительный интервал: $(53.72, 58.52)$
* FMeasure: $51.43$, доверительный интервал: $(49.23, 53.63)$

Видим, что на тесте качество достаточно ощутимо падает.

*Замечание* 

До определенного момента в модели была логика по частичному отбору кандидатов на этапе работы position selector. Если оставлять лишь 5 кандидатов, то в этом месте удавалось добиться FMeasure в районе $56.18$, что гораздо лучше чем сейчас. Тем не менее, при использовании более сложной модели ранжирования кандидатов, лучше оставить их всех до ранжирования.

## Выводы

1. Пока результат выглядит не очень хорошо, особенно смущает precision. Казалось, что BERT сможет более качественно выполнять исправления. Возможно, эта проблема решится если использовать какую-то модель ошибок и скоры из position selector для окончательного выбора кандидата.
2. Пробовал заменить модель в position selector на 3-граммную модель, которая обучалась без текстов из соц. сетей &mdash; результат примерно такой же.
3. Пробовал заменить модель в position selector на 4-граммную модель &mdash; результат примерно такой же.
4. Пробовал увеличить максимальное расстояние Дамера-Левенштейна до двух &mdash; результат стал сильно хуже.
5. Пробовал изменить аггрегирующую функцию по WordPiece-токенам в candidate scorer &mdash; результат стал немного хуже.